In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [3]:
import os
import numpy as np
import pandas as pd

In [17]:
sub0 = pd.read_csv('/content/drive/MyDrive/trending_submission.csv').sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv('/content/drive/MyDrive/exp004_submission.csv').sort_values('customer_id').reset_index(drop=True)
sub2 = pd.read_csv('/content/drive/MyDrive/by_cust_age__.csv').sort_values('customer_id').reset_index(drop=True)
# sub3 = pd.read_csv('/content/drive/MyDrive/submission.csv').sort_values('customer_id').reset_index(drop=True)

sub0.shape, sub1.shape, sub2.shape
# sub0.shape, sub1.shape
# sub0.shape, sub2.shape

((1371980, 2), (1371980, 2), (1371980, 2))

In [ ]:
# How many predictions are in common between models

print((sub0['prediction']==sub1['prediction']).mean())
print((sub0['prediction']==sub2['prediction']).mean())
print((sub1['prediction']==sub2['prediction']).mean())

0.0
0.0
7.288735987405064e-06


In [ ]:
print( sub0.head() )
print()
print( sub1.head() )
print()
print( sub2.head() )

                                         customer_id  \
0  00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...   
1  0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...   
2  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   
3  00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...   
4  00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...   

                                          prediction  
0  0568601043 0568601006 0656719005 0745232001 07...  
1  0826211002 0800436010 0739590027 0723529001 08...  
2  0794321007 0852643001 0852643003 0858883002 07...  
3  0448509014 0573085028 0751471001 0706016001 06...  
4  0730683050 0791587015 0896152002 0818320001 09...  

                                         customer_id  \
0  00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...   
1  0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...   
2  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   
3  00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...   
4  00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...

In [18]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
# sub0['prediction3'] = sub3['prediction']
del sub1, sub2
# del sub1
# del sub2
sub0.head()

,customer_id,prediction0,prediction1,prediction2,prediction3
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0745232001 07...,0751471001 0918292001 0706016001 0751471043 09...,0568601043 0568601006 0745232001 0751471001 0...,0915526001 0751471001 0918292001 0568601043 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0723529001 08...,0751471001 0918292001 0706016001 0751471043 09...,0826211002 0739590027 0811835004 0764280001 07...,0915526001 0751471001 0918292001 0706016001 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0751471001 0918292001 0706016001 07...,0794321007 0858883002 0852643003 0727808007 08...,0794321007 0915526001 0751471001 0918292001 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...,0751471001 0918292001 0706016001 0751471043 09...,0751471001 0579541001 0573085028 0673677002 06...,0915526001 0751471001 0918292001 0706016001 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...,0751471001 0918292001 0706016001 0751471043 09...,0730683050 0791587015 0896152002 0927530004 08...,0915526001 0751471001 0918292001 0706016001 07...


In [19]:
def cust_blend(dt, W = [1,1,1]):
# def cust_blend(dt, W = [1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]
    
    #Create a list of all model predictions
    REC = []
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    REC.append(dt['prediction3'].split())
    
    #Create a dictionary of items recommended. 
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))
    
    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    # Return the top 12 itens only
    return ' '.join(res[:12])

sub0['prediction'] = sub0.apply(cust_blend, W = [0.95, 1.15, 0.95], axis=1)
# 0.00233
# sub0['prediction'] = sub0.apply(cust_blend, W = [0.95, 1.15, 0.95], axis=1)
sub0.head()

,customer_id,prediction0,prediction1,prediction2,prediction3,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0745232001 07...,0751471001 0918292001 0706016001 0751471043 09...,0568601043 0568601006 0745232001 0751471001 0...,0915526001 0751471001 0918292001 0568601043 07...,0568601043 0751471001 0568601006 0918292001 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0723529001 08...,0751471001 0918292001 0706016001 0751471043 09...,0826211002 0739590027 0811835004 0764280001 07...,0915526001 0751471001 0918292001 0706016001 07...,0826211002 0751471001 0918292001 0739590027 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0751471001 0918292001 0706016001 07...,0794321007 0858883002 0852643003 0727808007 08...,0794321007 0915526001 0751471001 0918292001 07...,0794321007 0751471001 0858883002 0852643001 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...,0751471001 0918292001 0706016001 0751471043 09...,0751471001 0579541001 0573085028 0673677002 06...,0915526001 0751471001 0918292001 0706016001 07...,0751471001 0448509014 0706016001 0918292001 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...,0751471001 0918292001 0706016001 0751471043 09...,0730683050 0791587015 0896152002 0927530004 08...,0915526001 0751471001 0918292001 0706016001 07...,0730683050 0751471001 0791587015 0896152002 09...


In [14]:
# How many predictions are in common with ensemble

print((sub0['prediction']==sub0['prediction0']).mean())
print((sub0['prediction']==sub0['prediction1']).mean())
print((sub0['prediction']==sub0['prediction2']).mean())
# print((sub0['prediction']==sub0['prediction3']).mean())

0.0
0.0
0.0


In [20]:
del sub0['prediction0']
del sub0['prediction1']
del sub0['prediction2']
# del sub0['prediction3']
sub0.to_csv('submission-blend-1.csv', index=False)

In [8]:
# kaggleのAPIを使えるようにする
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes


In [9]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 2.1 MB 976 kB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 KB 537.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72858 sha256=7bc7e26bd147aad1692582409541d7dc87c9156b07c3b13471a09f27f3017d55
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [21]:
!kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f submission-blend-1.csv -m test

100% 258M/258M [00:10<00:00, 26.4MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations